In [ ]:
### Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

# 1. Load data hasil Tahap 1
df = pd.read_csv('../Data/Wind_Data_Cleaned.csv', parse_dates=['Timestamp'])
df = df.set_index('Timestamp')

# 2. Membuat Fitur Lag (t-1 sampai t-6)
# Membantu model melihat trend jangka sangat pendek
for i in range(1, 7):
    df[f'speed_lag_{i}'] = df['Wind Speed'].shift(i)

# 3. Membuat Fitur Waktu Siklik (Cyclical Features)
# Mengubah Jam dan Bulan menjadi Sin/Cos agar model paham pola harian/musiman
df['hour_sin'] = np.sin(2 * np.pi * df.index.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df.index.hour / 24)
df['month_sin'] = np.sin(2 * np.pi * (df.index.month - 1) / 12)
df['month_cos'] = np.cos(2 * np.pi * (df.index.month - 1) / 12)

# 4. Membuat Label/Target (Shift Negatif untuk masa depan)
df['target_3h'] = df['Wind Speed'].shift(-3)
df['target_6h'] = df['Wind Speed'].shift(-6)

# 5. Menghapus baris yang memiliki NaN akibat shifting
# (Baris awal tidak punya lag, baris akhir tidak punya target)
df_features = df.dropna()

# 6. Simpan dataset final untuk Training Model
df_features.to_csv('../Data/Wind_Data_Features.csv')

print("Feature Engineering Selesai!")
print(f"Jumlah kolom sekarang: {len(df_features.columns)}")
print(df_features[['Wind Speed', 'speed_lag_1', 'target_3h', 'target_6h']].head())

# Tahap 2: Feature Engineering (Rekayasa Fitur)
Proyek: Benchmarking Random Forest vs XGBoost - Prediksi Kecepatan Angin Stamet Pattimura Ambon

## 1. Pendahuluan
Pada tahap ini, dataset yang telah dibersihkan diubah menjadi format **Supervised Learning**. Karena model *Tree-based* (Random Forest dan XGBoost) tidak secara otomatis mengenali urutan waktu, kita perlu mengekstrak informasi temporal ke dalam bentuk kolom-kolom baru (fitur) agar model memiliki "konteks" masa lalu untuk memprediksi masa depan.

## 2. Fitur Lag (Memory Features)
Fitur Lag memberikan informasi kepada model mengenai apa yang terjadi pada jam-jam sebelumnya. Ini adalah prediktor terkuat dalam data deret waktu (*time series*).
* **Implementasi**: Kita membuat 6 fitur lag (*speed_lag_1* hingga *speed_lag_6*).
* **Kegunaan**: Memungkinkan model untuk melihat tren jangka pendek dan fluktuasi angin yang baru saja terjadi sebelum jam prediksi.

## 3. Fitur Siklik (Cyclical Time Features)
Waktu bersifat siklik (berulang). Angka jam 23:00 sebenarnya sangat dekat dengan jam 00:00, namun secara numerik (23 vs 0) keduanya terlihat sangat jauh.
* **Transformasi Sin/Cos**: Kita mengubah fitur **Jam** dan **Bulan** menggunakan fungsi Sinus dan Cosinus.
* **Kegunaan**: Memastikan model memahami hubungan periodik harian (Angin Darat/Laut) dan musiman (Monsun) secara kontinu.

## 4. Penentuan Target (Horizon Prediksi)
Sesuai dengan tujuan projek, kita menyiapkan dua skenario prediksi jangka pendek:
1. **target_3h**: Kecepatan angin pada 3 jam ke depan (t+3).
2. **target_6h**: Kecepatan angin pada 6 jam ke depan (t+6).

## 5. Pembersihan Akhir (Handling NaNs)
Proses shifting (pergeseran) data untuk membuat Lag dan Target akan menghasilkan nilai kosong (NaN) pada baris awal dan akhir dataset.
* **Tindakan**: Menghapus baris yang mengandung NaN untuk memastikan dataset yang masuk ke tahap pelatihan model bersifat padat dan lengkap.